This notebook is heavily adapted from the source below:

http://www.open3d.org/docs/release/tutorial/pipelines/colored_pointcloud_registration.html

In [1]:
import open3d as o3d
import numpy as np
import copy
import os
import sys


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Colored point cloud registration
This tutorial demonstrates an ICP variant that uses both geometry and color for registration. It implements the algorithm of [\[Park2017\]](../reference.html#park2017). The color information locks the alignment along the tangent plane. Thus this algorithm is more accurate and more robust than prior point cloud registration algorithms, while the running speed is comparable to that of ICP registration. This tutorial uses notations from [ICP registration](../pipelines/icp_registration.ipynb).

## Helper visualization function
In order to demonstrate the alignment between colored point clouds, `draw_registration_result_original_color` renders point clouds with their original color.

In [2]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target],
                                      zoom=0.5,
                                      front=[-0.2458, -0.8088, 0.5342],
                                      lookat=[1.7745, 2.2305, 0.9787],
                                      up=[0.3109, -0.5878, -0.7468])

## Input
The code below reads a source point cloud and a target point cloud from two files. An identity matrix is used as initialization for the registration.

In [16]:
print("1. Load two point clouds and show initial pose")

#source = o3d.io.read_point_cloud('/home/farshad/Documents/Cotton Imaging/18_17/Top Camera/18_17_top_Clean.ply')
#target = o3d.io.read_point_cloud('/home/farshad/Documents/Cotton Imaging/18_17/Bottom Camera/18_17_bot _Clean.ply')

target = o3d.io.read_point_cloud('/home/farshad/Documents/RTAB-Map/18_17_top_Clean.ply')
source = o3d.io.read_point_cloud('/home/farshad/Documents/RTAB-Map/18_17_bot _Clean.ply')

# draw initial alignment
# current_transformation = np.identity(4)

current_tranformation = np.array([[1, 0, 0, 0],
                                  [0, 1, 0, 0.15],
                                  [0, 0, 1, 0],
                                  [0, 0, 0, 1]])

# current_tranformation = np.array([[1, 0, 0, 0],
#                                   [0, 1, 0, 0.15],
#                                   [0, 0, 1, 0],
#                                   [0, 0, 0, 1]])

#draw_registration_result_original_color(source, target, current_transformation)
print(source)
print(target)

1. Load two point clouds and show initial pose
PointCloud with 22170906 points.
PointCloud with 17337453 points.


## Colored point cloud registration
The core function for colored point cloud registration is `registration_colored_icp`. Following [\[Park2017\]](../reference.html#park2017), it runs ICP iterations (see [Point-to-point ICP](../pipelines/icp_registration.ipynb#Point-to-point-ICP) for details) with a joint optimization objective

\begin{equation}
E(\mathbf{T}) = (1-\delta)E_{C}(\mathbf{T}) + \delta E_{G}(\mathbf{T})
\end{equation}

where $\mathbf{T}$ is the transformation matrix to be estimated. $E_{C}$ and $E_{G}$ are the photometric and geometric terms, respectively. $\delta\in[0,1]$ is a weight parameter that has been determined empirically.

The geometric term $E_{G}$ is the same as the [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) objective

\begin{equation}
E_{G}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathcal{K}$ is the correspondence set in the current iteration. $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$.

The color term $E_{C}$ measures the difference between the color of point $\mathbf{q}$ (denoted as $C(\mathbf{q})$) and the color of its projection on the tangent plane of $\mathbf{p}$.

\begin{equation}
E_{C}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big(C_{\mathbf{p}}(\mathbf{f}(\mathbf{T}\mathbf{q})) - C(\mathbf{q})\big)^{2},
\end{equation}

where $C_{\mathbf{p}}(\cdot)$ is a precomputed function continuously defined on the tangent plane of $\mathbf{p}$. Function$\mathbf{f}(\cdot)$ projects a 3D point to the tangent plane. For more details, refer to [\[Park2017\]](../reference.html#park2017).

To further improve efficiency, [\[Park2017\]](../reference.html#park2017) proposes a multi-scale registration scheme. This has been implemented in the following script.

In [17]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
#voxel_radius = [0.04, 0.02, 0.01]
voxel_radius = [0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
max_iter = [15000, 10000, 5000, 3500, 1500, 1000]

print("3. Colored point cloud registration")
for scale in range(6):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)
#draw_registration_result_original_color(source, target,
#                                        result_icp.transformation)

3. Colored point cloud registration
[15000, 0.06, 0]
3-1. Downsample with a voxel size 0.06
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=6.709556e-01, inlier_rmse=3.251229e-02, and correspondence_set size of 3911
Access transformation to get result.
[10000, 0.05, 1]
3-1. Downsample with a voxel size 0.05
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=6.163793e-01, inlier_rmse=2.794446e-02, and correspondence_set size of 5005
Access transformation to get result.
[5000, 0.04, 2]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=5.430218e-01, inlier_rmse=2.236283e-02, and correspondence_set size of 6595
Access transformation to get result.
[3500, 0.03, 3]
3-1. Downsample with a voxel size 0.03
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness

In [8]:
current_transformation

array([[ 0.99125198,  0.13185895, -0.00572154, -0.27515386],
       [-0.13154371,  0.99055784,  0.038619  ,  0.07375823],
       [ 0.01075978, -0.03752853,  0.99923763, -0.1889511 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In total, 3 layers of multi-resolution point clouds are created with `voxel_down_sample`. Normals are computed with vertex normal estimation. The core registration function `registration_colored_icp` is called for each layer, from coarse to fine. `lambda_geometric` is an optional argument for `registration_colored_icp` that determines $\lambda \in [0, 1]$ in the overall energy $\lambda E_{G} + (1-\lambda) E_{C}$.

The output is a tight alignment of the two point clouds. Notice the green triangles on the wall.

## Make a combined point cloud
`PointCloud` has a convenience operator `+` that can merge two point clouds into a single one. In the code below, the points are uniformly resampled using `voxel_down_sample` after merging. This is recommended post-processing after merging point clouds since it can relieve duplicated or over-densified points.

In [18]:
source_temp = copy.deepcopy(source)
source_temp.transform(result_icp.transformation)
pcd_combined = source_temp + target
#pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=0.002)
o3d.io.write_point_cloud('/home/farshad/Documents/RTAB-Map/18_17_combined_1.ply', pcd_combined)

True